### 10.3.1 分词工具

In [1]:
%%writefile a.txt

去参观 100 v

Overwriting a.txt


In [2]:
import jieba
print(' '.join(jieba.cut('今天我去参观展览馆', cut_all=True))) # 全模式
print(' '.join(jieba.cut('今天我去参观展览馆', cut_all=False))) # 精确模式

jieba.load_userdict('a.txt')
print(jieba.lcut('今天我去参观展览馆'))

import jieba.posseg as pseg
words = pseg.cut("今天我去参观展览馆")
for w in words:
    print("%s %s" %(w.word, w.flag))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.998 seconds.
Prefix dict has been built succesfully.


今天 我 去 参观 观展 展览 展览馆
今天 我 去 参观 展览馆
['今天', '我', '去参观', '展览馆']
今天 t
我 r
去参观 v
展览馆 n


### 10.3.2 TF-IDF逆文本频率指数

In [3]:
import jieba
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer  

arr = ['第一天我参观了美术馆',
       '第二天我参观了博物馆',
       '第三天我参观了动物园',]

arr = [' '.join(jieba.lcut(i)) for i in arr] # 分词
print(arr)

['第一天 我 参观 了 美术馆', '第二天 我 参观 了 博物馆', '第三天 我 参观 了 动物园']


In [4]:
vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(arr) 
word = vectorizer.get_feature_names() 
df = pd.DataFrame(X.toarray(), columns=word)
print(df)

   动物园  博物馆  参观  第一天  第三天  第二天  美术馆
0    0    0   1    1    0    0    1
1    0    1   1    0    0    1    0
2    1    0   1    0    1    0    0


In [5]:
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
weight = tfidf.toarray()
for i in range(len(weight)): # 访问每一句
    print("第{}句：".format(i))
    for j in range(len(word)):  # 访问每个词
        if weight[i][j] > 0.05:  # 只显示重要关键字
            print(word[j],round(weight[i][j],2))  # 保留两位小数

第0句：
参观 0.39
第一天 0.65
美术馆 0.65
第1句：
博物馆 0.65
参观 0.39
第二天 0.65
第2句：
动物园 0.65
参观 0.39
第三天 0.65


In [6]:
# 写程序实现TF-IDF方法

from collections import Counter
import numpy as np

countlist = []
for i in range(len(arr)):
    count = Counter(arr[i].split(' ')) # 用空格将字串切分成字符串列表，统计每个词出现次数
    countlist.append(count)
print(countlist)

def tf(word, count): 
    return count[word] / sum(count.values())
def contain(word, count_list): # 统计包含关键词word的句子数量
    return sum(1 for count in count_list if word in count)
def idf(word, count_list):
    return np.log(len(count_list) / (contain(word, count_list)) + 1)  #为避免分母为0，分母加1
def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)
for i, count in enumerate(countlist):
    print("第{}句：".format(i))
    scores = {word: tfidf(word, count, countlist) for word in count}
    for word, score in scores.items():
        print(word, round(score, 2))

[Counter({'第一天': 1, '我': 1, '参观': 1, '了': 1, '美术馆': 1}), Counter({'第二天': 1, '我': 1, '参观': 1, '了': 1, '博物馆': 1}), Counter({'第三天': 1, '我': 1, '参观': 1, '了': 1, '动物园': 1})]
第0句：
第一天 0.28
我 0.14
参观 0.14
了 0.14
美术馆 0.28
第1句：
第二天 0.28
我 0.14
参观 0.14
了 0.14
博物馆 0.28
第2句：
第三天 0.28
我 0.14
参观 0.14
了 0.14
动物园 0.28
